In [0]:
import requests
import json

import pandas as pd
import numpy as np

In [0]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json][timeout:25]; area(3600301091)->.searchArea; ( node['amenity'='hospital'](area.searchArea); node['amenity'='school'](area.searchArea);relation['admin_level'='10'](area.searchArea); ); out body; >; out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [0]:
df = pd.DataFrame(data['elements'])

In [0]:
df.head()

,center,id,lat,lon,members,nodes,tags,type
0,NaN,501170977,-5.869024,-35.234268,NaN,NaN,{'amenity': 'school'},node
1,NaN,501170997,-5.823610,-35.222611,NaN,NaN,"{'amenity': 'school', 'name': 'Centro de Atenç...",node
2,NaN,501171016,-5.871019,-35.221614,NaN,NaN,"{'amenity': 'school', 'name': 'Piaget'}",node
3,NaN,501619315,-5.816246,-35.204844,NaN,NaN,"{'amenity': 'hospital', 'name': 'Hospital da U...",node
4,NaN,501784871,-5.812943,-35.210726,NaN,NaN,{'amenity': 'school'},node


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770 entries, 0 to 2769
Data columns (total 8 columns):
center     407 non-null object
id         2770 non-null int64
lat        2327 non-null float64
lon        2327 non-null float64
members    36 non-null object
nodes      407 non-null object
tags       499 non-null object
type       2770 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 173.2+ KB


In [0]:
df['tags']

0                                   {'amenity': 'school'}
1       {'amenity': 'school', 'name': 'Centro de Atenç...
2                 {'amenity': 'school', 'name': 'Piaget'}
3       {'amenity': 'hospital', 'name': 'Hospital da U...
4                                   {'amenity': 'school'}
5       {'amenity': 'school', 'name': 'Escola Boa Ideia'}
6                                 {'amenity': 'hospital'}
7                                   {'amenity': 'school'}
8             {'amenity': 'school', 'name': 'SEST/SENAT'}
9                                   {'amenity': 'school'}
10                                  {'amenity': 'school'}
11                   {'amenity': 'school', 'name': 'CEI'}
12      {'amenity': 'hospital', 'name': 'UMS Cidade Sa...
13                                  {'amenity': 'school'}
14         {'amenity': 'school', 'name': 'Contemporâneo'}
15                                  {'amenity': 'school'}
16      {'addr:city': 'Natal - RN', 'addr:housenumber'...
17      {'addr

In [0]:
def get_amenity(row):
  try:
    amenity = row['amenity']
    return amenity
  except:
    return np.nan
  
def get_name(row):
  try:
    name = row['name']
    return name
  except:
    return np.nan

In [0]:
df['amenity'] = df.tags.apply(get_amenity)
df['name'] = df.tags.apply(get_name)

In [0]:
df['amenity'].unique()

array(['school', 'hospital', nan, 'bench'], dtype=object)

In [0]:
df.head()

,center,id,lat,lon,members,nodes,tags,type,amenity,name
0,NaN,501170977,-5.869024,-35.234268,NaN,NaN,{'amenity': 'school'},node,school,NaN
1,NaN,501170997,-5.823610,-35.222611,NaN,NaN,"{'amenity': 'school', 'name': 'Centro de Atenç...",node,school,Centro de Atenção Integral a Criança e ao Adol...
2,NaN,501171016,-5.871019,-35.221614,NaN,NaN,"{'amenity': 'school', 'name': 'Piaget'}",node,school,Piaget
3,NaN,501619315,-5.816246,-35.204844,NaN,NaN,"{'amenity': 'hospital', 'name': 'Hospital da U...",node,hospital,Hospital da UNIMED
4,NaN,501784871,-5.812943,-35.210726,NaN,NaN,{'amenity': 'school'},node,school,NaN


In [0]:
# load the GeoJSON data and use 'UTF-8'encoding
geo_json_natal = json.load(open('natal.geojson'))